In [9]:
import numpy as np

from ts_modelling import data_producer
from ts_modelling.simple_experiment import SimpleExp
from argparse import Namespace
import yaml
with open('args.yaml', 'r') as file:
    config = yaml.safe_load(file)
args = Namespace(**config)
exp = SimpleExp(args)

from data_producer import DataProducer
dp = DataProducer(length=5000, n_vars=1, path=args.root_path)
dp.add_sine(freq_amp=[(1,2)])
dp.add_trend(3, 0.05)


Use CPU


array([[   1.1224373 ],
       [   1.44799839],
       [   2.02092694],
       ...,
       [-111.83351051],
       [-110.77576082],
       [-111.0180385 ]])

'./data'